
# Синтаксический анализ: SpaCy

В этом задании мы познакомимся с библиотекой для обработки текстов [SpaCy](https://spacy.io), с помощью которой можно выполнять анализ текстов на разных уровнях.

Здесь, в основном, мы будем работать с синтаксическим анализом, который также уже есть в SpaCy.

Сначала давайте установим библитеку:

In [1]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


Теперь давайте посмотрим, что собой представляют тексты и токены здесь:

In [2]:
import spacy

nlp = spacy.load('en') # here we load model
doc = nlp('Hello     World!')
for token in doc:
    print('"' + token.text + '"')


"Hello"
"    "
"World"
"!"


Каждый токен содержит информацию о своей позиции (на уровне символов) в предложении:

In [3]:
doc = nlp(u'Hello     World!')
for token in doc:
    print('"' + token.text + '"', token.idx)

"Hello" 0
"    " 6
"World" 10
"!" 15


Токен содержит многие атрибуты:

In [5]:
doc = nlp("Next week I'll   be in Madrid.")
for token in doc:
    print("{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\t{7}".format(
        token.text,
        token.idx,
        token.lemma_,
        token.is_punct,
        token.is_space,
        token.shape_,
        token.pos_,
        token.tag_
    ))

Next	0	next	False	False	Xxxx	ADJ	JJ
week	5	week	False	False	xxxx	NOUN	NN
I	10	-PRON-	False	False	X	PRON	PRP
'll	11	will	False	False	'xx	VERB	MD
  	15	  	False	True	  	SPACE	_SP
be	17	be	False	False	xx	AUX	VB
in	20	in	False	False	xx	ADP	IN
Madrid	23	Madrid	False	False	Xxxxx	PROPN	NNP
.	29	.	True	False	.	PUNCT	.


SpaCy также умеет находить границы предложений в тексте:

In [6]:
doc = nlp("These are apples. These are oranges.")
 
for sent in doc.sents:
    print(sent)

These are apples.
These are oranges.


Частеречная разметка:

In [7]:
doc = nlp("Next week I'll be in Madrid.")
print([(token.text, token.tag_) for token in doc])

[('Next', 'JJ'), ('week', 'NN'), ('I', 'PRP'), ("'ll", 'MD'), ('be', 'VB'), ('in', 'IN'), ('Madrid', 'NNP'), ('.', '.')]


Chunking (нахождение синтаксических групп в предложении, только именных):

In [19]:
doc = nlp("Wall Street Journal just published an interesting piece on crypto currencies")
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.label_, chunk.root.text)

Wall Street Journal NP Journal
an interesting piece NP piece
crypto currencies NP currencies


Dependency parsing:

In [9]:
doc = nlp('Wall Street Journal just published an interesting piece on crypto currencies')
 
for token in doc:
    print("{0}/{1} <--{2}-- {3}/{4}".format(
        token.text, token.tag_, token.dep_, token.head.text, token.head.tag_))

Wall/NNP <--compound-- Street/NNP
Street/NNP <--compound-- Journal/NNP
Journal/NNP <--nsubj-- published/VBD
just/RB <--advmod-- published/VBD
published/VBD <--ROOT-- published/VBD
an/DT <--det-- piece/NN
interesting/JJ <--amod-- piece/NN
piece/NN <--dobj-- published/VBD
on/IN <--prep-- piece/NN
crypto/NNP <--compound-- currencies/NNS
currencies/NNS <--pobj-- on/IN


Визуализация дерева зависимостей:


In [10]:
from spacy import displacy
 
doc = nlp(u'Wall Street Journal just published an interesting piece on crypto currencies')
displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

Теперь загрузим CoNLL корпус, который содержит синтаксически размеченные предложения:

In [13]:
import nltk

nltk.download('conll2000')

[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.


True

`sents` содержит предложения, и каждое предложение представляет собой массив строк (каждая строка - токен).

`chunked_sents` содержит chunks каждого предложения.

In [14]:
from nltk.corpus import conll2000

sents = conll2000.sents()
chunked_sents = conll2000.chunked_sents()

print(len(sents))
print(len(chunked_sents))

10948
10948


In [15]:
print(sents[0])
print(' '.join(sents[0]))
print(chunked_sents[0])

['Confidence', 'in', 'the', 'pound', 'is', 'widely', 'expected', 'to', 'take', 'another', 'sharp', 'dive', 'if', 'trade', 'figures', 'for', 'September', ',', 'due', 'for', 'release', 'tomorrow', ',', 'fail', 'to', 'show', 'a', 'substantial', 'improvement', 'from', 'July', 'and', 'August', "'s", 'near-record', 'deficits', '.']
Confidence in the pound is widely expected to take another sharp dive if trade figures for September , due for release tomorrow , fail to show a substantial improvement from July and August 's near-record deficits .
(S
  (NP Confidence/NN)
  (PP in/IN)
  (NP the/DT pound/NN)
  (VP is/VBZ widely/RB expected/VBN to/TO take/VB)
  (NP another/DT sharp/JJ dive/NN)
  if/IN
  (NP trade/NN figures/NNS)
  (PP for/IN)
  (NP September/NNP)
  ,/,
  due/JJ
  (PP for/IN)
  (NP release/NN)
  (NP tomorrow/NN)
  ,/,
  (VP fail/VB to/TO show/VB)
  (NP a/DT substantial/JJ improvement/NN)
  (PP from/IN)
  (NP July/NNP and/CC August/NNP)
  (NP 's/POS near-record/JJ deficits/NNS)
  ./.

# Задание 1


Посчитайте частотность синтаксических отношений в корпусе, используя SpaCy для разметки. Выведите их на экран: от самых частотных к менее частотным.

In [ ]:
# your code

# Задание 2


Узнайте, насколько хорошо SpaCy справляется с извлечением chunks, а именно с извлечением именных групп!

Используя корпус, посчитайте точность, полноту и F1-меру (будет лучше, если вы сами напишите метрики, но вы также можете использовать функции из других библиотек).

In [ ]:
# your code